## Dataframe operations
Spark DataFrames allow operations similar to pandas Dataframes. We demonstrate some of those.

For more, see the [official guide](https://spark.apache.org/docs/latest/sql-programming-guide.html) and [this article](https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/)

## But first, some important details
Spark on datahub can run in two modes:

* **local mode** which means that it is run on the same computer as the head node. This is convenient for small jobs and for debugging. Can also be done on your laptop.

* **remote mode** in which the head node and the worker nodes are separate. This mode requires that the spark cluster is up and running. In this case the full resources of the clusters are available. This mode is useful when processing large jobs.

In [2]:
import os
import sys

import pyspark
from pyspark import SparkContext
from lib import sparkConfig

10.37.128.30


In [3]:
%%time
#sc.stop()  # uncomment if sparkContex already exists
sc = SparkContext(conf=sparkConfig.conf)
sc

CPU times: user 31 ms, sys: 21.1 ms, total: 52 ms
Wall time: 2.96 s


<SparkContext master=spark://spark-master:7077 appName=spark on MLBP>

In [4]:
%%time
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType,BinaryType

sqlContext = SQLContext(sc)
sqlContext

CPU times: user 3.2 ms, sys: 2.15 ms, total: 5.35 ms
Wall time: 54.6 ms


In [5]:
%%time
parquet_path='/datasets/cs255-sp22-a00-public/2020-parquet'
df=sqlContext.read.parquet(parquet_path)

CPU times: user 1.48 ms, sys: 3.64 ms, total: 5.12 ms
Wall time: 3.85 s


In [6]:
df.printSchema()

root
 |-- Station: string (nullable = true)
 |-- Measurement: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Values: binary (nullable = true)



In [7]:
%%time
print(df.count())
df.show(1)

182102
+-----------+-----------+----+--------------------+
|    Station|Measurement|Year|              Values|
+-----------+-----------+----+--------------------+
|AGM00060355|       TMIN|2020|[42 00 44 00 39 0...|
+-----------+-----------+----+--------------------+
only showing top 1 row

CPU times: user 4.61 ms, sys: 591 µs, total: 5.2 ms
Wall time: 4 s


### .describe()
The method `df.describe()` computes five statistics for each column of the dataframe `df`.

The statistics are: **count, mean, std, min,max**

You get the following man page using the command `df.describe?`

```
Signature: df.describe(*cols)
Docstring:
Computes statistics for numeric and string columns.

This include count, mean, stddev, min, and max. If no columns are
given, this function computes statistics for all numerical or string columns.

.. note:: This function is meant for exploratory data analysis, as we make no
    guarantee about the backward compatibility of the schema of the resulting DataFrame.

>>> df.describe(['age']).show()
+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|                 2|
|   mean|               3.5|
| stddev|2.1213203435596424|
|    min|                 2|
|    max|                 5|
+-------+------------------+
>>> df.describe().show()
+-------+------------------+-----+
|summary|               age| name|
+-------+------------------+-----+
|  count|                 2|    2|
|   mean|               3.5| null|
| stddev|2.1213203435596424| null|
|    min|                 2|Alice|
|    max|                 5|  Bob|
+-------+------------------+-----+

.. versionadded:: 1.3.1
File:      ~/spark-2.2.1-bin-hadoop2.7/python/pyspark/sql/dataframe.py
Type:      method
```

In [8]:
df.describe().select('Station','Measurement','Year').show() 

+-----------+-----------+------+
|    Station|Measurement|  Year|
+-----------+-----------+------+
|     182102|     182102|182102|
|       null|       null|2020.0|
|       null|       null|   0.0|
|AE000041196|       AWDR|  2020|
|ZI000067983|       WT18|  2020|
+-----------+-----------+------+



#### groupby and agg
The method `.groupby(col)` groups rows according the value of the column `col`.  
The method `.agg(spec)` computes a summary for each group as specified in `spec`

In [9]:
df.groupby('Measurement').agg({'Year': 'min', 'Station':  'count'}).show()

+-----------+--------------+---------+
|Measurement|count(Station)|min(Year)|
+-----------+--------------+---------+
|       WESD|          6658|     2020|
|       PGTM|           267|     2020|
|       AWDR|            50|     2020|
|       WT07|            39|     2020|
|       SX33|            14|     2020|
|       EVAP|           216|     2020|
|       WT10|            12|     2020|
|       SN53|             7|     2020|
|       SN35|             4|     2020|
|       TMIN|         13830|     2020|
|       DATX|           164|     2020|
|       MDPR|         12756|     2020|
|       WT09|           159|     2020|
|       SX51|             6|     2020|
|       DATN|           167|     2020|
|       WT05|          1112|     2020|
|       SN36|             2|     2020|
|       SN32|           145|     2020|
|       SN31|            19|     2020|
|       SX52|            67|     2020|
+-----------+--------------+---------+
only showing top 20 rows



In [ ]:
# THis command will load the python module that defines the SQL functions
#%load ls ~/spark-latest/python/pyspark/sql/functions.py

### Using SQL queries on DataFrames

There are two main ways to manipulate  DataFrames:

#### Imperative manipulation
Using python methods such as `.select` and `.groupby`.
* Advantage: order of operations is specified.
* Disrdavantage : You need to describe both **what** is the result you want and **how** to get it.

#### Declarative Manipulation (SQL)
* Advantage: You need to describe only **what** is the result you want.
* Disadvantage: SQL does not have primitives for common analysis operations such as **covariance**

### Using sql commands on a dataframe
Spark supports a [subset](https://spark.apache.org/docs/latest/sql-programming-guide.html#supported-hive-features) of the Hive SQL query language.

For example, You can use [Hive `select` syntax](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Select) to select a subset of the rows in a dataframe.

To use sql on a dataframe you need to first `register` it as a `TempTable`.

for variety, we are using here a small dataframe loaded from a JSON file.

#### Counting the number of occurances of each measurement, imparatively

In [10]:
%%time
L=df.groupBy('measurement').count().collect()
#L is a list of Rows (collected DataFrame)

CPU times: user 4.08 ms, sys: 6.55 ms, total: 10.6 ms
Wall time: 2.19 s


In [11]:
D=[(e.measurement,e['count']) for e in L]
sorted(D,key=lambda x:x[1], reverse=True)[:6]

[('PRCP', 39952),
 ('SNOW', 23266),
 ('SNWD', 18635),
 ('TMIN', 13830),
 ('TMAX', 13770),
 ('DAPR', 12825)]

In [12]:
sorted(D,key=lambda x:x[1], reverse=False)[:6]

[('WT18', 1), ('SN36', 2), ('SX56', 2), ('SN56', 2), ('SX36', 3), ('SN35', 4)]

#### Counting the number of occurances of each measurement, declaratively.

In [13]:
parquet_path='/datasets/cs255-sp22-a00-public/2020-parquet'
!ls -ld $parquet_path

drwxr-xr-x 2 yfreund nogroup 78 Apr 11 19:15 /datasets/cs255-sp22-a00-public/2020-parquet


In [18]:
df = sqlContext.read.parquet(parquet_path)

In [22]:
df.show(4)

+-----------+-----------+----+--------------------+
|    Station|Measurement|Year|              Values|
+-----------+-----------+----+--------------------+
|AGM00060355|       TMIN|2020|[42 00 44 00 39 0...|
|AGM00060511|       TMIN|2020|[FB FF D9 FF F0 F...|
|AGM00060531|       TAVG|2020|[57 00 51 00 60 0...|
|AGM00060557|       TMIN|2020|[D4 FF D0 FF E6 F...|
+-----------+-----------+----+--------------------+
only showing top 4 rows



## Registrering a dataframe as a table in a database

In order to apply SQL commands to a dataframe, it has to first be registered as a table in the database managed by sqlContext.

In [14]:
sqlContext.registerDataFrameAsTable(df,'weather') #using older sqlContext instead of newer (V2.0) sparkSession

In [24]:
%%time
query="""
SELECT measurement,COUNT(measurement) AS count,
                   MIN(year) AS MinYear 
FROM weather  
GROUP BY measurement 
ORDER BY count
"""
print(query)
sqlContext.sql(query).show(5)


SELECT measurement,COUNT(measurement) AS count,
                   MIN(year) AS MinYear 
FROM weather  
GROUP BY measurement 
ORDER BY count

+-----------+-----+-------+
|measurement|count|MinYear|
+-----------+-----+-------+
|       WT18|    1|   2020|
|       SN36|    2|   2020|
|       SX56|    2|   2020|
|       SN56|    2|   2020|
|       SX36|    3|   2020|
+-----------+-----+-------+
only showing top 5 rows

CPU times: user 4.76 ms, sys: 2.97 ms, total: 7.73 ms
Wall time: 1.23 s


#### Performing a map command
* In order to perform a `map` on a dataframe, you first need to transform it into an RDD.

* **Not** the recommended way. Better way is to use built-in sparkSQL functions.
* Or register new ones (Advanced).

In [18]:
df.rdd.map(lambda row:(row.Station,row.Year)).take(5)

[('AGM00060355', 2020),
 ('AGM00060511', 2020),
 ('AGM00060531', 2020),
 ('AGM00060557', 2020),
 ('AGM00060602', 2020)]

#### Aggregations 
* **Aggregation** can be used, in combination with built-in sparkSQL functions 
to compute statistics of a dataframe.
* computation will be fast thanks to combined optimzations with database operations.

* A partial list : `count(), approx_count_distinct(), avg(), max(), min()`

* Of these, the interesting one is `approx_count_distinct()` which uses sampling to get an approximate count fast.

* [The gory details](http://spark.apache.org/docs/2.2.0/api/python/_modules/pyspark/sql/functions.html)

In [ ]:
import pyspark.sql.functions as F # used here just for show.

In [ ]:
df.agg({'station':'approx_count_distinct'}).show()

#### Approximate Quantile

* Suppose we want to partition the years into 10 ranges
* such that in each range we have approximately the same number of records.
* The method `.approxQuantile` will use a sample to do this for us.

In [19]:
print('with accuracy 0.1: ',df.approxQuantile('Year', [0.1*i for i in range(1,10)], 0.1))
print('with accuracy 0.01: ',df.approxQuantile('Year', [0.1*i for i in range(1,10)], 0.01))

with accuracy 0.1:  [2020.0, 2020.0, 2020.0, 2020.0, 2020.0, 2020.0, 2020.0, 2020.0, 2020.0]
with accuracy 0.01:  [2020.0, 2020.0, 2020.0, 2020.0, 2020.0, 2020.0, 2020.0, 2020.0, 2020.0]


### Reading rows selectively
Suppose we are only interested in snow measurements. We can apply an SQL query directly to the 
parquet files. As the data is organized in columnar structure, we can do the selection efficiently without loading the whole file to memory.

Here the file is small, but in real applications it can consist of hundreds of millions of records. In such cases loading the data first to memory and then filtering it is very wasteful.

In [23]:
query="""SELECT station,measurement,year 
FROM parquet.`%s` 
WHERE measurement=\"SNOW\" """%parquet_path
print(query)
df2 = sqlContext.sql(query)
print(df2.count(),df2.columns)
df2.show(5)

SELECT station,measurement,year 
FROM parquet.`/datasets/cs255-sp22-a00-public/2020-parquet` 
WHERE measurement="SNOW" 
23266 ['station', 'measurement', 'year']
+-----------+-----------+----+
|    station|measurement|year|
+-----------+-----------+----+
|CA001045100|       SNOW|2020|
|CA001062544|       SNOW|2020|
|CA001063298|       SNOW|2020|
|CA001096468|       SNOW|2020|
|CA00109E7R6|       SNOW|2020|
+-----------+-----------+----+
only showing top 5 rows



## Summary

* Dataframes can be manipulated decleratively, which allows for more optimization.
* Dataframes can be stored and retrieved from Parquet files.
* It is possible to refer directly to a parquet file in an SQL query.
* See you next time!

## References
* For an introduction to Spark SQL and Dataframes see: [Spark SQL, DataFrames](https://spark.apache.org/docs/latest/sql-programming-guide.html#spark-sql-dataframes-and-datasets-guide)
* Also [spark-dataframe-and-operations](https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/) from [analyticsvidhya.com](https://www.analyticsvidhya.com)

For complete API reference see
* [SQL programming guide](https://spark.apache.org/docs/latest/sql-programming-guide.html) For Java, Scala and Python (Implementation is first in Scala and Python, later pyspark)
* [pyspark API for the DataFrame class](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame) 
* [pyspark API for the pyspark.sql module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module)
